## **Installing Libraries**

# New Section

In [ ]:
!pip install beautifulsoup4
!pip install requests
!pip install azure-storage-blob
!pip install google-cloud-storage
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.6/408.6 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 6.8 MB/s eta 0:00:00


### **Importing Libraries**

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import json
import re
import os
import time
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from io import StringIO
from azure.storage.blob import BlobServiceClient, ContainerClient
import pandas as pd
from io import BytesIO
from typing import Container
import calendar

## **Extracting Data from BLOB**

In [ ]:
AZURE_STORAGE_CONNECTION_STRING = "DefaultEndpointsProtocol=https;AccountName=storhv;AccountKey=nmRWurX4P/PaRa9gCSFt5m56/opYaGtjRd3Qu97SKn5w5VxJzMW7l7FWFhQpi+Lc3TqM3GOAbVbP+AStWH1diw==;EndpointSuffix=core.windows.net"
CONTAINER_NAME = "hvor"

# Initialize BlobServiceClient and ContainerClient
blob_service_client = BlobServiceClient.from_connection_string(AZURE_STORAGE_CONNECTION_STRING)
container_client = blob_service_client.get_container_client(CONTAINER_NAME)

In [ ]:
# Step 1: List blobs in the container
print("Listing blobs in the container:")
blobs = container_client.list_blobs()
for blob in blobs:
    print(blob.name)

# Step 2: Specify the name of the uploaded CSV
csv_filename = "cleaned_data_v3.csv"
blob_client = container_client.get_blob_client(blob=csv_filename)

# Step 3: Download the CSV file
print(f"Downloading {csv_filename} from Blob Storage...")
download_stream = blob_client.download_blob()
csv_data = BytesIO(download_stream.readall())

# Step 4: Load into a pandas DataFrame
df = pd.read_csv(csv_data)
print("Data loaded successfully. Here's a preview:")
print(df.head())

Listing blobs in the container:
cleaned_data_v3.csv
Data loaded successfully. Here's a preview:
      CAMIS                         DBA CUISINE DESCRIPTION_yelp  \
0  50126924  walnut bus stop restaurant                 American   
1  50126924  walnut bus stop restaurant                 American   
2  50126924  walnut bus stop restaurant                 American   
3  50126924  walnut bus stop restaurant                 American   
4  50126924  walnut bus stop restaurant                 American   

  INSPECTION DATE_yelp                                      ACTION_yelp  \
0            4/10/2023  Violations were cited in the following area(s).   
1            4/10/2023  Violations were cited in the following area(s).   
2            4/10/2023  Violations were cited in the following area(s).   
3            4/10/2023  Violations were cited in the following area(s).   
4            4/10/2023  Violations were cited in the following area(s).   

  VIOLATION CODE_yelp                       

In [ ]:
df.head()


,CAMIS,DBA,CUISINE DESCRIPTION_yelp,INSPECTION DATE_yelp,ACTION_yelp,VIOLATION CODE_yelp,VIOLATION DESCRIPTION_yelp,CRITICAL FLAG_yelp,SCORE_yelp,GRADE_yelp,...,RECORD DATE_inspection,INSPECTION TYPE_inspection,Latitude_inspection,Longitude_inspection,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,50126924,walnut bus stop restaurant,American,4/10/2023,Violations were cited in the following area(s).,04L,Evidence of mice or live mice in establishment...,Critical,13.0,A,...,12/3/2024,Pre-permit (Operational) / Initial Inspection,40.800344,-73.91067,201.0,8.0,1900.0,2097591.0,2.025940e+09,BX39
1,50126924,walnut bus stop restaurant,American,4/10/2023,Violations were cited in the following area(s).,04L,Evidence of mice or live mice in establishment...,Critical,13.0,A,...,12/3/2024,Cycle Inspection / Re-inspection,40.800344,-73.91067,201.0,8.0,1900.0,2097591.0,2.025940e+09,BX39
2,50126924,walnut bus stop restaurant,American,4/10/2023,Violations were cited in the following area(s).,04L,Evidence of mice or live mice in establishment...,Critical,13.0,A,...,12/3/2024,Cycle Inspection / Re-inspection,40.800344,-73.91067,201.0,8.0,1900.0,2097591.0,2.025940e+09,BX39
3,50126924,walnut bus stop restaurant,American,4/10/2023,Violations were cited in the following area(s).,04L,Evidence of mice or live mice in establishment...,Critical,13.0,A,...,12/3/2024,Pre-permit (Operational) / Re-inspection,40.800344,-73.91067,201.0,8.0,1900.0,2097591.0,2.025940e+09,BX39
4,50126924,walnut bus stop restaurant,American,4/10/2023,Violations were cited in the following area(s).,04L,Evidence of mice or live mice in establishment...,Critical,13.0,A,...,12/3/2024,Cycle Inspection / Initial Inspection,40.800344,-73.91067,201.0,8.0,1900.0,2097591.0,2.025940e+09,BX39


## **Dim Location**

In [ ]:
dim_location = df[['Latitude_inspection', 'Longitude_inspection']].drop_duplicates().reset_index(drop=True)
dim_location['location_ID'] = dim_location.index + 1
dim_location.head()

,Latitude_inspection,Longitude_inspection,location_ID
0,40.800344,-73.910670,1
1,40.801980,-73.910314,2
2,40.802314,-73.920705,3
3,40.803201,-73.912913,4
4,40.803692,-73.920050,5


In [ ]:
# Create Dim_Location
dim_location = df[['CAMIS', 'Latitude_inspection', 'Longitude_inspection']].drop_duplicates().reset_index(drop=True)

# Rename columns to match the model
dim_location.rename(columns={
    'CAMIS': 'location_ID',  # location_ID
    'Latitude_inspection': 'latitude',
    'Longitude_inspection': 'longitude'
}, inplace=True)

dim_location = dim_location[['location_ID', 'latitude', 'longitude']]
dim_location.head()

,location_ID,latitude,longitude
0,50126924,40.800344,-73.910670
1,50079722,40.801980,-73.910314
2,50158855,40.802314,-73.920705
3,50128864,40.803201,-73.912913
4,50154142,40.803692,-73.920050


## **Dim Restaurant**

In [ ]:
# Create Dim_Restaurant
dim_restaurant = df[['CAMIS', 'DBA', 'CUISINE DESCRIPTION_yelp', 'Community Board', 'Council District', 'NTA']].drop_duplicates()
dim_restaurant.rename(columns={
    'CAMIS': 'restaurant_ID',
    'DBA': 'restaurant_name',
    'CUISINE DESCRIPTION_yelp': 'cuisine_type',
    'Community Board': 'community_board',
    'Council District': 'council_district',
    'NTA': 'borough'
}, inplace=True)

dim_restaurant.head()

,restaurant_ID,restaurant_name,cuisine_type,community_board,council_district,borough
0,50126924,walnut bus stop restaurant,American,201.0,8.0,BX39
21,50079722,the bronx brewery-tap room,American,201.0,8.0,BX39
29,50158855,"fresh direct,",NaN,201.0,8.0,BX39
30,50128864,estrella del caribe restaurant bar,Latin American,201.0,8.0,BX39
55,50154142,kandys palace,NaN,201.0,8.0,BX39


## **Dim Inspection Date**

In [ ]:
# Create Dim_Inspection_Date Table
dim_inspection_date = df[['INSPECTION DATE_yelp', 'INSPECTION TYPE_yelp', 'GRADE_yelp', 'GRADE DATE_yelp', 'RECORD DATE_inspection']].drop_duplicates().reset_index(drop=True)
dim_inspection_date.rename(columns={
    'INSPECTION DATE_yelp': 'inspection_date',
    'INSPECTION TYPE_yelp': 'inspection_type',
    'GRADE_yelp': 'grade',
    'GRADE DATE_yelp': 'grade_date',
    'RECORD DATE_inspection': 'record_date'
}, inplace=True)

# Add a surrogate key for the date dimension
dim_inspection_date['date_key'] = dim_inspection_date.index + 1

dim_inspection_date = dim_inspection_date[['date_key', 'inspection_date', 'inspection_type', 'grade', 'grade_date', 'record_date']]
dim_inspection_date.head()

,date_key,inspection_date,inspection_type,grade,grade_date,record_date
0,1,4/10/2023,Pre-permit (Operational) / Re-inspection,A,4/10/2023,12/3/2024
1,2,1/11/2023,Cycle Inspection / Initial Inspection,A,1/11/2023,12/3/2024
2,3,1/1/1900,NaN,NaN,NaN,12/3/2024
3,4,4/3/2023,Pre-permit (Non-operational) / Initial Inspection,NaN,NaN,12/3/2024
4,5,6/20/2024,Administrative Miscellaneous / Initial Inspection,NaN,NaN,12/3/2024


## **Fact Restaurant Analysis**

In [ ]:
# Create Facts Restaurant Analysis
facts_restaurant_analysis = df[['CAMIS', 'Latitude_inspection', 'Longitude_inspection',
                                'INSPECTION DATE_yelp', 'VIOLATION CODE_yelp',
                                'SCORE_yelp', 'GRADE_yelp', 'VIOLATION DESCRIPTION_yelp']].copy()

facts_restaurant_analysis.rename(columns={
    'CAMIS': 'restaurant_ID',
    'Latitude_inspection': 'latitude',
    'Longitude_inspection': 'longitude',
    'INSPECTION DATE_yelp': 'inspection_date',
    'VIOLATION CODE_yelp': 'violation_code',
    'SCORE_yelp': 'yelp_rating',
    'GRADE_yelp': 'grade',
    'VIOLATION DESCRIPTION_yelp': 'violation_description'
}, inplace=True)

# Add location_ID from Dim_Location
facts_restaurant_analysis = facts_restaurant_analysis.merge(dim_location,
                                                            left_on=['restaurant_ID'],
                                                            right_on=['location_ID'],
                                                            how='left')
facts_restaurant_analysis.head()

,restaurant_ID,latitude_x,longitude_x,inspection_date,violation_code,yelp_rating,grade,violation_description,location_ID,latitude_y,longitude_y
0,50126924,40.800344,-73.91067,4/10/2023,04L,13.0,A,Evidence of mice or live mice in establishment...,50126924,40.800344,-73.91067
1,50126924,40.800344,-73.91067,4/10/2023,04L,13.0,A,Evidence of mice or live mice in establishment...,50126924,40.800344,-73.91067
2,50126924,40.800344,-73.91067,4/10/2023,04L,13.0,A,Evidence of mice or live mice in establishment...,50126924,40.800344,-73.91067
3,50126924,40.800344,-73.91067,4/10/2023,04L,13.0,A,Evidence of mice or live mice in establishment...,50126924,40.800344,-73.91067
4,50126924,40.800344,-73.91067,4/10/2023,04L,13.0,A,Evidence of mice or live mice in establishment...,50126924,40.800344,-73.91067


## **Saving Tables**

In [ ]:
# Referenced "https://stackoverflow.com/questions/50014827/write-python-dataframe-as-csv-into-azure-blob" for csv to blob upload

dim_location.to_csv('dim_location.csv', index=False)
dim_restaurant.to_csv('dim_restaurant.csv', index=False)
dim_inspection_date.to_csv('dim_inspection_date.csv', index=False)
facts_restaurant_analysis.to_csv('facts_restaurant_analysis.csv', index=False)

# Uploads DataFrame as a CSV to Azure Blob Storage
def upload_to_blob(df, container_client, blob_name):
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=False)
    csv_buffer.seek(0)
    blob_client = container_client.get_blob_client(blob=blob_name)
    blob_client.upload_blob(csv_buffer.getvalue(), overwrite=True)
    print(f"{blob_name} uploaded to Blob Storage.")

upload_to_blob(dim_location, container_client, "dim_location.csv")
upload_to_blob(dim_restaurant, container_client, "dim_restaurant.csv")
upload_to_blob(dim_inspection_date, container_client, "dim_inspection_date.csv")
upload_to_blob(facts_restaurant_analysis, container_client, "facts_restaurant_analysis.csv")

print("Tables saved successfully.")

dim_location.csv uploaded to Blob Storage.
dim_restaurant.csv uploaded to Blob Storage.
dim_inspection_date.csv uploaded to Blob Storage.
facts_restaurant_analysis.csv uploaded to Blob Storage.
Tables saved successfully.
